# NLP Tutorial: Section 3

In [28]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode


In [30]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers=classifiers
    
    def classify(self, features):
        vote=[c.classify(features) for c in self._classifiers]
        print ("vote",vote)
        print ("mode",mode(vote))
        return vote,mode(vote)
            
    def confidence(self,features):
        vote=self.classify(features)[0]
        conf=vote.count(self.classify(features)[1])/len(vote)
        #conf2=len(self.classify(features)[1])/len(vote)
        print ("conf",conf)
        #print ("conf2",conf2)
        return conf
    

In [31]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

all_words = nltk.FreqDist([w.lower() for w in movie_reviews.words()])
word_features = list(all_words.keys())[:3000]

def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features


featuresets = [(find_features(rev), category) for (rev, category) in documents]
        
training_set = featuresets[:1900]
testing_set =  featuresets[1900:]

#classifier = nltk.NaiveBayesClassifier.train(training_set)

classifier_f = open("naivebayes.pickle","rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)


LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

voted_classifier = VoteClassifier(classifier,
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)


Original Naive Bayes Algo accuracy percent: 82.0
Most Informative Features
                   sucks = True              neg : pos    =     11.9 : 1.0
              schumacher = True              neg : pos    =     10.5 : 1.0
           unimaginative = True              neg : pos    =      8.5 : 1.0
                  turkey = True              neg : pos    =      7.9 : 1.0
             silverstone = True              neg : pos    =      7.8 : 1.0
                  annual = True              pos : neg    =      7.5 : 1.0
                 frances = True              pos : neg    =      7.5 : 1.0
                 idiotic = True              neg : pos    =      7.4 : 1.0
                  stinks = True              neg : pos    =      7.1 : 1.0
                  wasted = True              neg : pos    =      6.7 : 1.0
                  inject = True              neg : pos    =      6.7 : 1.0
                    cube = True              neg : pos    =      6.5 : 1.0
                  alicia 

/Users/saijananiganesan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 77.0
LinearSVC_classifier accuracy percent: 74.0
NuSVC_classifier accuracy percent: 77.0
vote ['pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'pos']
mode pos
vote ['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg']
mode neg
vote ['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg']
mode neg
vote ['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
mode pos
vote ['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg']
mode neg
vote ['neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'neg']
mode neg
vote ['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
mode pos
vote ['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
mode pos
vote ['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg']
mode neg
vote ['pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'neg']
mode pos
vote ['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg']
mode neg
vote ['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg']
mode neg
vote ['pos', 'pos', 'neg', 'pos', 'pos', 'pos', 'neg']
mode pos
vote ['neg', 'neg', 'neg', 'neg', 'n